In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
base_path = './drive/My Drive/Colab Notebooks/Speech Denoising/LotOfData/'

base_path_train = base_path + 'tr/'
base_path_val = base_path + 'v/'
base_path_test = base_path + 'te/'

base_path_pickle = base_path + 'pickle/'

base_path_result = base_path + 'result/'

In [3]:
!pip install librosa

In [0]:
import librosa
import pickle

In [0]:
import os
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [0]:
trx = []
trs = []
trn = []

trx_val = []
trs_val = []
trn_val = []
target = []

trx_len = []

max_width = 513
max_length = 200

In [8]:
# check if pickle files exist
pickle_paths = ['trx.pickle', 'trs.pickle', 'trn.pickle', 'trx_val.pickle', \
                'trs_val.pickle', 'trn_val.pickle', 'target.pickle', 
                'trx_len.pickle', 'trx_id.pickle']
found_saved = all([os.path.exists(base_path_pickle + x) for x in pickle_paths])
found_saved

True

In [0]:
if found_saved:
  with open(base_path_pickle + 'trx.pickle', 'rb') as f:
    trx = pickle.load(f)
  with open(base_path_pickle + 'trs.pickle', 'rb') as f:
    trs = pickle.load(f)
  with open(base_path_pickle + 'trn.pickle', 'rb') as f:
    trn = pickle.load(f)
  with open(base_path_pickle + 'trx_val.pickle', 'rb') as f:
    trx_val = pickle.load(f)
  with open(base_path_pickle + 'trs_val.pickle', 'rb') as f:
    trs_val = pickle.load(f)
  with open(base_path_pickle + 'trn_val.pickle', 'rb') as f:
    trn_val = pickle.load(f)
  with open(base_path_pickle + 'target.pickle', 'rb') as f:
    target = pickle.load(f)
  with open(base_path_pickle + 'trx_len.pickle', 'rb') as f:
    trx_len = pickle.load(f)
  with open(base_path_pickle + 'trx_id.pickle', 'rb') as f:
    trx_id = pickle.load(f)

In [0]:
def get_stft(file_path):
  s, sr = librosa.load(file_path, sr=None)
  stft = librosa.stft(s, n_fft=1024, hop_length=512).T
  stft_val = np.zeros((max_length, max_width))
  stft_val[:stft.shape[0], :stft.shape[1]] = np.abs(stft)
  return stft, stft_val

In [0]:
def load_from_directory(directory, file_prefix, all_required):
  file_prefix_dirty = file_prefix + 'x'
  file_prefix_clean = file_prefix + 's'
  file_prefix_noise = file_prefix + 'n'
  
  lfd_x = []
  lfd_s = []
  lfd_n = []
  
  lfd_x_val = []
  lfd_s_val = []
  lfd_n_val = []
  
  lfd_target = []
  lfd_len = []
  lfd_id = []
  
  n = 0
  for file in sorted(os.listdir(directory)):
    # consider only .wav files starting with file_prefix_dirty
    if file.endswith('.wav') and file.startswith(file_prefix_dirty):
      if n == 20:
        break
      n += 1
      
      file_id = file[len(file_prefix_dirty):-len('.wav')]

      dirty_file_path = os.path.join(directory, file)

      if all_required:
        # check if there is corresponding target/clean file
        clean_file_name = file.replace(file_prefix_dirty, file_prefix_clean)
        clean_file_path = os.path.join(directory, clean_file_name)
        if not os.path.exists(clean_file_path):
          continue

        noise_file_name = file.replace(file_prefix_dirty, file_prefix_noise)
        noise_file_path = os.path.join(directory, noise_file_name)
        if not os.path.exists(clean_file_path):
          continue

      # load both dirty, clean and noise files
      train_dirty, train_dirty_val = get_stft(dirty_file_path)
      if all_required:
        train_clean, train_clean_val = get_stft(clean_file_path)
        train_noise, train_noise_val = get_stft(noise_file_path)
      
      lfd_x.append(train_dirty)
      lfd_len.append(len(train_dirty))
      lfd_id.append(file_id)
      
      if all_required:
        lfd_s.append(train_clean)
        lfd_n.append(train_noise)

      lfd_x_val.append(train_dirty_val)
      if all_required:
        lfd_s_val.append(train_clean_val)
        lfd_n_val.append(train_noise_val)

      if all_required:
        # multiply by 1 to convert boolean to integer
        lfd_target.append(1 * (train_clean_val > train_noise_val)) 
        
  if all_required:
    return lfd_x, lfd_s, lfd_n, np.array(lfd_x_val), np.array(lfd_s_val), \
                                np.array(lfd_n_val), np.array(lfd_target), \
                                np.array(lfd_len), np.array(lfd_id)
  return lfd_x, np.array(lfd_x_val), np.array(lfd_len), np.array(lfd_id)

In [64]:
if  found_saved:
  trx, trs, trn, trx_val, trs_val, trn_val, target, trx_len, trx_id = load_from_directory(base_path_train, \
                                                                        'tr', True)

16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000
16000


KeyboardInterrupt: ignored

In [13]:
trx_val.shape, trs_val.shape, trn_val.shape, target.shape

((20, 200, 513), (20, 200, 513), (20, 200, 513), (20, 200, 513))

In [41]:
trx_id

array(['0000', '0001', '0002', '0003', '0004', '0005', '0006', '0007',
       '0008', '0009', '0010', '0011', '0012', '0013', '0014', '0015',
       '0016', '0017', '0018', '0019'], dtype='<U4')

In [0]:
if not found_saved:
  with open(base_path_pickle + 'trx.pickle', 'wb') as f:
    pickle.dump(trx, f)
  with open(base_path_pickle + 'trs.pickle', 'wb') as f:
    pickle.dump(trs, f)
  with open(base_path_pickle + 'trn.pickle', 'wb') as f:
    pickle.dump(trn, f)
  with open(base_path_pickle + 'trx_val.pickle', 'wb') as f:
    pickle.dump(trx_val, f)
  with open(base_path_pickle + 'trs_val.pickle', 'wb') as f:
    pickle.dump(trs_val, f)
  with open(base_path_pickle + 'trn_val.pickle', 'wb') as f:
    pickle.dump(trn_val, f)
  with open(base_path_pickle + 'target.pickle', 'wb') as f:
    pickle.dump(target, f)
  with open(base_path_pickle + 'trx_len.pickle', 'wb') as f:
    pickle.dump(trx_len, f)
  with open(base_path_pickle + 'trx_id.pickle', 'wb') as f:
    pickle.dump(trx_len, f)

In [0]:
num_samples_tr = len(trx_val)
batch_size = 10
num_features = 513
num_hidden = 256

learning_rate = 0.001
num_epochs = 10

In [0]:
X = tf.placeholder(tf.float32, [None, max_length, num_features])
Y = tf.placeholder(tf.float32, [None, max_length, num_features])
# xlen = tf.placeholder(tf.int32, None)

In [18]:
cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_hidden))
output, state = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
dense_1 = tf.layers.Dense(units=513, activation=tf.nn.sigmoid)(output)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.


In [19]:
dense_1

<tf.Tensor 'dense/Sigmoid:0' shape=(?, 200, 513) dtype=float32>

In [20]:
# calculate loss - only calculate loss on valid data
loss = tf.losses.mean_squared_error(labels=Y, predictions=dense_1)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss=loss)
init = tf.global_variables_initializer()

Instructions for updating:
Use tf.cast instead.


In [0]:
sess = tf.Session()
saver = tf.train.Saver()

In [23]:
sess.run(tf.global_variables_initializer())

for epoch in range(num_epochs):
  loss_val = 0
  for i in range(0, num_samples_tr, batch_size):
    start_idx = i
    end_idx = min(i + batch_size, num_samples_tr)
    
    batch_x = trx_val[start_idx:end_idx]
    batch_y = target[start_idx:end_idx]
    
    _, lv = sess.run([train, loss], feed_dict={X: batch_x, Y: batch_y})
    loss_val += lv
  print(epoch, loss_val)

0 0.4985465407371521
1 0.48801906406879425
2 0.46282438933849335
3 0.38196876645088196
4 0.32473835349082947
5 0.2945317029953003
6 0.28038884699344635
7 0.2747050002217293
8 0.27046943455934525
9 0.26420432329177856


In [0]:
def clean(X, M, filename):
  cleaned = X * M
  sh_test = librosa.istft(cleaned, hop_length=512)
    
  # Save to a file
  librosa.output.write_wav(filename, sh_test, 16000)
  print('Saved to ', filename)

In [0]:
tex, tex_val, tex_len, tex_id = load_from_directory(base_path_test, 'te', False)

In [70]:
num_samples_te = len(tex_val)

for i in range(0, num_samples_te, batch_size):
  start_idx = i
  end_idx = min(i + batch_size, num_samples_te)
  
  batch_x = tex_val[start_idx:end_idx]

  m_pred = sess.run([dense_1], feed_dict={X: batch_x})
  for j in range(start_idx, end_idx):
    x = tex[j]
    m = m_pred[0][i - start_idx][:tex_len[j], :]
    fname = base_path_result + 'cleaned' + tex_id[j] + '.wav'
    
    clean(x, m, fname)
    break

Saved to  ./drive/My Drive/Colab Notebooks/Speech Denoising/LotOfData/result/cleaned0000.wav
Saved to  ./drive/My Drive/Colab Notebooks/Speech Denoising/LotOfData/result/cleaned0010.wav


In [44]:
tex[0].shape

(110, 513)